# Assignment 4
### Name: Zimu Li
### PID: A13897206

Github: https://github.com/zil244/DSC106_SP21

In [25]:
import altair as alt
from vega_datasets import data
import pandas as pd
import numpy as np

### Problem 1

In [26]:
df = pd.read_csv('flunet2010_11countries_106.csv')
source = df
countries = list(source.columns[1:])
dc = {'week':[],'country':[],'count':[]}
for i in range(len(source['week'])):
    for j in range(1,len(countries) + 1):
        dc['week'].append(i+1)
        dc['count'].append(source.iloc[i,j])
        dc['country'].append(countries[j-1])
source_1 = pd.DataFrame(dc)
source_1.head(5)

,week,country,count
0,1,Afghanistan,5.0
1,1,Argentina,4.0
2,1,Australia,2.0
3,1,Canada,41.0
4,1,China,2179.0


In [27]:
base = alt.Chart(source_1).mark_line().transform_filter(
    alt.FieldOneOfPredicate(
        field='country', oneOf=countries
    )).encode(x='week',y='count')
dropdown = alt.binding_select(options=countries)
country_select = alt.selection_single(
    fields=['country'], bind=dropdown, name="select"
)
color_condition = alt.condition(country_select,
                      alt.Color('country:N', legend=None),
                      alt.value('lightgray'))
base.add_selection(country_select).encode(color=color_condition)

alt.Chart(...)

#### Bounus

In [28]:
source_2 = source_1.dropna().astype(
    {'week':'int64','country':'O','count':'int64'}
)
c1 = pd.DataFrame({'country': countries})
selection = alt.selection_multi(fields=['country'],toggle="true")
color = alt.condition(selection, alt.Color(
    'country:N'), alt.value('lightgrey'))
c1_selector = alt.Chart(c1).mark_rect().encode(
    y='country', color=color
).add_selection(selection)
chart = alt.Chart(source_2).mark_line().encode(
    x='week', y=alt.Y('count', scale=alt.Scale(domain=[0, 2500])), color='country'
).transform_filter(selection)

c1_selector | chart

alt.HConcatChart(...)

### Problem 2

In [29]:
c2 = pd.read_csv('countries.csv')

In [30]:
c3 = list(source_1.groupby('country').sum().index)
cnt = list(source_1.groupby('country').sum()['count'])
df1 = pd.DataFrame({'country':c3,'count':cnt})
cnt = []
for i in range(len(c2['name'])):
    b1 = False
    for j in range(len(df1['country'])):
        if c2['name'][i] == df1['country'][j]:
            cnt.append(df1['count'][j])
            b1 = True
    if b1 == False:
        cnt.append(np.NaN)
c2['count'] = cnt

In [31]:
sphere = alt.sphere()
graticule = alt.graticule()
world_countries = alt.topo_feature(data.world_110m.url, 'countries')
alt.layer(
    alt.Chart(sphere).mark_geoshape(fill='lightgrey'),
    alt.Chart(graticule).mark_geoshape(stroke='white', strokeWidth=0.5),
    alt.Chart(world_countries).mark_geoshape(fill='grey', stroke='black'),
    alt.Chart(world_countries).mark_geoshape().encode(
        color='count:Q',
        tooltip=['count:Q']
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(c2, 'id', ['count'])
    )
).project(
    'naturalEarth1')

alt.LayerChart(...)